In [1]:
import pandas as pd
import numpy as np
import json
from tqdm.notebook import tqdm
from functools import partial

import multiprocessing
from multiprocessing import Process, Manager

import warnings

warnings.filterwarnings(action='ignore')

data_dir = './Data/'

In [2]:
song_meta_df = pd.read_json(data_dir + 'song_meta_data_v3.json')
song_meta_df = song_meta_df.sort_values('id')
song_meta_df = song_meta_df.reset_index(drop = True)
song_meta_df['song_embedding_idx'] = song_meta_df.index

In [3]:
with open(data_dir + 'song_id2playlist_wmc.json', 'r') as f:
    new_data = json.load(f)

In [4]:
song_id_li = song_meta_df['id'].tolist()

# 상위 Top10 Ranking

In [5]:
def get_top10_song_id2playlist(song_id, data):
    w_li = data[str(song_id) + 'w']
    m_li = data[str(song_id) + 'm']
    c_li = data[str(song_id) + 'c']
    
    w_cnt = 3
    m_cnt = 3
    c_cnt = 3
    
    method = 1
    
    ret_li = set(w_li[: w_cnt + 1] + m_li[: m_cnt + 1] + c_li[: c_cnt + 1])
    
    while len(ret_li) < 10:
        if method == 1:
            c_cnt += 1
        elif method == 2:
            m_cnt += 1
        elif method == 3:
            w_cnt += 1
        
        ret_li = set(w_li[: w_cnt + 1] + m_li[: m_cnt + 1] + c_li[: c_cnt + 1])
        method += 1
        
        if method == 4:
            method = 1
    
    ret_li = list(ret_li)
    
    return ret_li

In [ ]:
top10_song_id2playlist = {}

for song_id in tqdm(song_id_li):
    top10_song_id2playlist[song_id] = get_top10_song_id2playlist(song_id = song_id, data = new_data)

In [ ]:
for song_id in tqdm(song_id_li):
    cnt = len(top10_song_id2playlist[song_id])
    if cnt != 10:
        print(song_id, cnt)

In [8]:
with open(data_dir + 'top10_song_id2playlist.json', 'w') as f:
    json.dump(top10_song_id2playlist, f)

# 순위 기반 Top10 Ranking

In [9]:
ranking_song_id2playlist = {}

def get_ranking_song_id2playlist(song_id, data):
    w_li = data[str(song_id) + 'w']
    m_li = data[str(song_id) + 'm']
    c_li = data[str(song_id) + 'c']
    
    score_li = np.array([1 / i for i in range(1, 51)])
    total_song_li = list(set(w_li + m_li + c_li))
    
    song_df = pd.DataFrame(index = total_song_li)
    song_df.loc[w_li, 'w_score'] = score_li
    song_df.loc[m_li, 'm_score'] = score_li
    song_df.loc[c_li, 'c_score'] = score_li
    song_df = song_df.fillna(0.02)
    
    song_df['total_score'] = song_df['w_score'] + song_df['m_score'] + song_df['c_score']
    song_df = song_df.sort_values('total_score', ascending = False)
    
    ret_li = song_df.index.tolist()[:10]
    
    return ret_li

In [ ]:
ranking_song_id2playlist = {}

for song_id in tqdm(song_id_li):
    ranking_song_id2playlist[song_id] = get_ranking_song_id2playlist(song_id = song_id, data = new_data)

In [ ]:
for song_id in tqdm(song_id_li):
    cnt = len(ranking_song_id2playlist[song_id])
    if cnt != 10:
        print(song_id, cnt)

In [12]:
with open(data_dir + 'ranking_song_id2playlist.json', 'w') as f:
    json.dump(ranking_song_id2playlist, f)